In [29]:
from langchain.llms.ollama import Ollama
from langchain.chat_models import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.memory import ConversationKGMemory

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

# choose chat
chat = ChatOllama(
    # model="gemma:latest",
    model="mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

def get_history():
    return memory.load_memory_variables({})
def add_message(i, o):
    memory.save_context({"input":i}, {"output":o})

## ConversationBuffer Memory
memory = ConversationBufferMemory(return_messages=False) #챗모델을 사용할거면 return_messages를 True로 설정
# memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
# print(get_history())

## ConversationBuffer Window Memory, 가장 최근 몇개만 저장
memory = ConversationBufferWindowMemory(return_messages=True, k=3)

# add_message(1,1)
# add_message(2,2)
# add_message(3,3)
# add_message(4,4)
# print(get_history())

## ConversationSummary Memory, 메세지가 많아지면 요약해서 토큰양을 줄이기 위해 이거 사용
memory = ConversationSummaryMemory(llm=chat)
# add_message("Hi I'm AJin, I live in South Korea", "Wow that is so cool!")
# add_message("South Korea is so pretty", "I wish I could go!!!")
# print(get_history())

## ConversationSummary Buffer Memory, 가장 최근거 몇개만 기억하고 그것을 넘어가면 요약해서 저장함
memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=50, return_messages=True)
# add_message("Hi I'm AJin, I live in South Korea", "Wow that is so cool!")
# add_message("South Korea is so pretty", "I wish I could go!!!")
# add_message("South Korea is colorful.", "I wish I could go! yes!")
# print(get_history())

## ConversationKGM(Knowledge Graph) Memory
memory = ConversationKGMemory(llm=chat, return_messages=True)
# add_message("Hi I'm AiJin, I live in South Korea", "Wow that is so cool!")
# print(memory.load_memory_variables({"input":"who is AiJin"}))


In [27]:
## Memory on LLMChain
memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=60, memory_key="chat_history")
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You: 
"""
chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=False # 체인 로그 확인용
)
# chain.predict(question="My name is JungWu.")
# chain.predict(question="I live in Seoul.")
# chain.predict(question="What is my name?")

## Chat Based Memory
memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=100,
    memory_key="chat_history",
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human. You speak briefly."),
    MessagesPlaceholder(variable_name="chat_history"), # 누가 보냈는지, 예측하기 어렵고 제한 없는 양의 메세지를 가져 각각 구분함
    ("human", "{question}")    
])
chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=prompt,
    verbose=False # 체인 로그 확인용
)
# chain.predict(question="My name is JungWu")
# chain.predict(question="What is my name?")

In [34]:
## LCEL(LangChain Expression Language) Based Memory, 메모리를 수동적으로 저장 및 사용, 추천
memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=100,
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human. You give short answers."),
    MessagesPlaceholder(variable_name="history"), # 누가 보냈는지, 예측하기 어렵고 제한 없는 양의 메세지를 가져 각각 구분함
    ("human", "{question}")    
])
def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]
chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat
def invoke_chain(question):
    result = chain.invoke({
        "question": question
    })
    memory.save_context({"input":question}, {"output":result.content})
    print(result)
invoke_chain("My name is Tina.")

{'question': 'My name is Tina.'}
 Hello Tina, nice to meet you. How can I assist you today?content=' Hello Tina, nice to meet you. How can I assist you today?'


In [32]:
invoke_chain("What is my name?")

{'question': 'What is my name?'}
 The name mentioned earlier was Tina. Is there a specific name you'd like me to use instead? Let me know if that's the case! Otherwise, feel free to ask any question or request assistance with a task. I'm here to help! 😊content=" The name mentioned earlier was Tina. Is there a specific name you'd like me to use instead? Let me know if that's the case! Otherwise, feel free to ask any question or request assistance with a task. I'm here to help! 😊"
